In [ ]:
#1) MOUNT THE GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#2) ACCESS THE DATABASE

import os
address="/content/drive/MyDrive/datasets/PlantVillageMini"

#list files
classes=os.listdir(address)
num_classes=len(classes)
print(f"Number of classes {num_classes}")

count=0
for class_name in classes:
  for _ in os.listdir(os.path.join(address,class_name)):
    count+=1

print(f"Number of images: {count}")

Number of classes 5
Number of images: 4627


In [ ]:
#3) CREATING A TENSORFLOW DATASET
import numpy as np
from sklearn.model_selection import train_test_split #For train test split
from tensorflow.keras.utils import to_categorical #For one hot encoding

#Get filepaths and their respective labels
data_dir=address

file_paths=list()
labels=list()

for class_id,class_name in enumerate(classes):
  for image_name in os.listdir(os.path.join(address,class_name)):
    file_paths.append(os.path.join(address,class_name,image_name))
    labels.append(class_id)     #Class ID is in integer form, 1,2,3,...

#Convert it into numpy array
file_paths=np.array(file_paths)
labels=np.array(labels)

#Train test split
X_train,X_test,y_train,y_test=train_test_split(file_paths,labels,test_size=0.2, random_state=42)

#One hot encoding
y_train=to_categorical(y_train, num_classes)
y_test=to_categorical(y_test, num_classes)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3701,)
(926,)
(3701, 5)
(926, 5)


In [ ]:
#4) LOADING AND PREPROCESSING IMAGES

import tensorflow as tf

print(f"Number of GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")
print(f"Number of CPUs Available: {len(tf.config.list_physical_devices('CPU'))}")

input_size=[224,224]

def preprocess_image(image_path, label):
  image=tf.io.read_file(image_path)
  image=tf.image.decode_jpeg(image, channels=3)
  image=tf.image.resize(image,input_size)
  image=image/255.0
  return image,label

#creating a tensorflow dataset
train_dataset=tf.data.Dataset.from_tensor_slices((X_train,y_train))
test_dataset=tf.data.Dataset.from_tensor_slices((X_test,y_test))

#apply preprocessing
train_dataset=train_dataset.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset=test_dataset.map(preprocess_image).batch(32)

Number of GPUs Available: 0
Number of CPUs Available: 1


In [ ]:
#5) LOADING THE PREPROCESSED MODEL

from tensorflow.keras import layers, models

#load mobilenetV2
base_model=tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,     #remove the last layer
    weights='imagenet'    #load weights trained on imagenet
)

#freeze base model
for layer in base_model.layers:
    layer.trainable = False

#define model
#adding our custom classification layer
model=models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),    #reduces feature map to a vector
    layers.Dense(128, activation="relu"),   #extra dense layer for better training
    layers.Dense(num_classes, activation="softmax")
])

#unfreeze last two layers
for layer in base_model.layers[-2:]:
    layer.trainable = True

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,597 (9.24 MB)

 Trainable params: 167,173 (653.02 KB)

 Non-trainable params: 2,255,424 (8.60 MB)

In [ ]:
#6) COMPILE THE MODEL

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

EPOCHS=10
#early_stop=EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss')   #stop training early if accuracy starts to reduce
reduce_lr=ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_lr=1e-6)

model.fit(train_dataset, validation_data=test_dataset, epochs=EPOCHS, callbacks=[reduce_lr])

Epoch 1/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 170s 1s/step - accuracy: 0.8148 - loss: 0.5182 - val_accuracy: 0.8996 - val_loss: 0.2441 - learning_rate: 0.0010
Epoch 2/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 183s 1s/step - accuracy: 0.9581 - loss: 0.1066 - val_accuracy: 0.9633 - val_loss: 0.0897 - learning_rate: 0.0010
Epoch 3/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 212s 1s/step - accuracy: 0.9852 - loss: 0.0566 - val_accuracy: 0.9708 - val_loss: 0.0735 - learning_rate: 0.0010
Epoch 4/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.9919 - loss: 0.0299 - val_accuracy: 0.9600 - val_loss: 0.1019 - learning_rate: 0.0010
Epoch 5/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 954ms/step - accuracy: 0.9912 - loss: 0.0296
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
116/116 ━━━━━━━━━━━━━━━━━━━━ 198s 1s/step - accuracy: 0.9912 - loss: 0.0296 - val_accuracy: 0.9752 - val_loss: 0.0766 - learning_rate: 0.0010
Epoch 6/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 209s 1s/step - accuracy: 0.9986 - loss: 0.

In [ ]:
#Testing for accuracy
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

29/29 ━━━━━━━━━━━━━━━━━━━━ 29s 982ms/step - accuracy: 0.9745 - loss: 0.0766
Test Loss: 0.06525544822216034
Test Accuracy: 98.06%


In [ ]:
model.save("LeafClassifier_MobileNet3_unfreeze_plus_layer.keras")
from google.colab import files
files.download('LeafClassifier_MobileNet3_unfreeze_plus_layer.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>